In [2]:
import numpy as np
import pandas as pd
import yfinance as yf # Yahoo Finance API
from datetime import datetime
from pprint import pprint # make printing data more readable
from treasury.client import FederalTreasuryClient # Fedearal Open Source Treasury API
import plotly.graph_objs as graph # just in case we need to graph
# ticker_symbol_input = yf.Ticker("MSFT")
# print(ticker_symbol_input.info)

In [ ]:
# parent/super class
class EDWARD_dashboard:
    def __init__(self, ticker_symbol: str,  contract_type: str, exercise_price: int, expiration_date: str):
        self.ticker_symbol = ticker_symbol.upper()
        self.current_date = datetime.now().strftime("%Y-%m-%d")
        self.contract_type = contract_type
        self.exercise_price = exercise_price
        self.expiration_date = expiration_date

    def days_between(self):
        d1, d2 = datetime.strptime(self.current_date, "%Y-%m-%d"), datetime.strptime(self.expiration_date, "%Y-%m-%d")
        diff_times = abs((d2 - d1).days)
        return diff_times

    def market_price(self):
        ticker_symbol = yf.Ticker(self.ticker_symbol).info
        market_price = ticker_symbol['regularMarketPrice']
        return market_price # spot price

    def risk_free_rate(self):
        # still need to use API to find nominal_rate and inflation_rate
        risk_free_rate = ((1+nominal_rate)/(1+inflation_rate))

    def dividend_yield(self):
        ticker_symbol = yf.Ticker(self.ticker_symbol).info
        dividend_rate = ticker_symbol['dividendRate']
        if dividend_rate == None:
            dividend_rate = 0
        return dividend_rate

    def options_data(self, type): # type can by calls or puts
        try:
            option_data = yf.Ticker(self.ticker_symbol).option_chain(date=self.expiration_date)
            print(f"Could find a matching expiration date for {self.ticker_symbol} {type} options")
        except ValueError:
            print(f"Expirations date chosen isn't available for this stock: {self.ticker_symbol} Here is list of Available {type} options ")
            option_data = yf.Ticker(self.ticker_symbol).option_chain()
        if type == 'calls':
            return option_data.calls
        else:
            return option_data.puts

    def futures_formula(self):
        Price_contract =  self.market_price()*np.e**((self.risk_free_rate()-self.dividend_yield())*(self.days_between()))
        return Price_contract

    def relationship_futuresprice_exepctedspotprice(self, k): #k is an investor’s required return for this investment. Ask  if formula is right
        # ask if this method is neccessary or repetitive of the futures_method and also incomplete method at the moment
        indicator = np.e**((self.risk_free_rate()-k) * self.days_between())
        if indicator == 0:
            return f"No systematic risk"
        elif indicator < 0:
            return f"Negative sytematic risk"
        elif indicator > 0:
            return f"Positive systematic risk"
        pass

    def portfolio(self):
        ticker_symbol = yf.Ticker(self.ticker_symbol)
        pass

    def __str__(self):
        return f"Current date {str(self.current_date)}; difference between dates {str(self.days_between())}; upper case ticker {self.ticker_symbol}; market {self.market_price()}; dividend rate {self.dividend_yield()}: "

tesla = EDWARD_dashboard("tSlA","future",300,"2022-11-11")
print(tesla)
# print(tesla.futures_formula())
# tesla.options_data('calls')
tesla.options_data('puts')


In [ ]:
# User Inputs
input_symbol = input("Ticker Symbol you would like to use? ")
# ticker_symbol = yf.Ticker(input_symbol)
input_contract_type = input("What contracct type would you like to use (Futures, Options, Swaps)? ")
input_exercise_price = int(input("What price would you like to exercise the shares at? "))
input_expiration_date = input("What date would you like to set the expiration? (Y-M-D)")
# EDWARD_dashboard(input_symbol, input_contract_type, input_exercise_price, input_expiration_date)
